<a href="https://colab.research.google.com/github/Galal-pic/Data-analysis-workshop-projects/blob/main/Random_Forest_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##`import Spacy and en_core_wb_small`

In [24]:
!pip install nltk --quiet
!pip install spacy==2.3.5 --quiet
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for spacy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for spacy
ERROR: Could not build wheels for spacy, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


##`install libs `

In [25]:
!pip install pytesseract --quiet
!pip install pyresparser --quiet
!pip install python-docx  --quiet

##`import libs`

In [1]:
#import normal libs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas  as pd
import ast
sns.set_style('darkgrid')
# import NLP libs
import nltk
import string
import re
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import spacy
spacy_model = spacy.load('en_core_web_sm')
# import Text exteact libs
from textblob import Word
#from pyresparser import ResumeParser
from docx import Document
# import Vectorization libs
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer
from sklearn.decomposition import TruncatedSVD
# import ML libs
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


##`import data from Drive`

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_skills = pd.read_csv('/content/drive/MyDrive/data/job_data_with_skills_v1.csv')
df_resume = pd.read_csv('/content/drive/MyDrive/data/UpdatedResumeDataSet.csv')
df_skills.head(5)

,Job_title,Job_link,Job_description,Keyword,skills,way_2_skills,combined
0,DevOps Engineer,https://eg.linkedin.com/jobs/view/devops-engin...,We looking DevOps Engineer join team develop s...,scrab_ddata,"['engineer', 'unwanted traffic', 'infrastructu...","['System', 'Analyze', 'Policies', 'Debugging',...","['engineer', 'unwanted traffic', 'infrastructu..."
1,DevOps Engineer,https://eg.linkedin.com/jobs/view/devops-engin...,Who We Are We owned funded one Egypt renowned ...,scrab_ddata,"['perspectives', 'strong focus scalability sec...","['Technical', 'Linux', 'Workflow', 'Lifecycle'...","['perspectives', 'strong focus scalability sec..."
2,DevOps Engineer,https://eg.linkedin.com/jobs/view/devops-engin...,Wind IS Company Oracle Partner Egypt KSA UAE h...,scrab_ddata,"['development', 'engineer', 'Job Requirements ...","['System', 'Website', 'Design', 'Engineering',...","['development', 'engineer', 'Job Requirements ..."
3,Cloud Infrastructure Engineer,https://eg.linkedin.com/jobs/view/cloud-infras...,Who We Are We owned funded one Egypt renowned ...,scrab_ddata,"['related', 'perspectives', 'big stay', 'suit'...","['Hardware', 'Dhcp', 'Linux', 'Cloud', 'Vmware...","['related', 'perspectives', 'big stay', 'suit'..."
4,DevOps Engineer,https://eg.linkedin.com/jobs/view/devops-engin...,About Axis We believe financial services acces...,scrab_ddata,['Jenkins CD Bambop CircleCI Git Action Gitlab...,"['Automation', 'Ansible', 'Docker', 'Security'...",['Jenkins CD Bambop CircleCI Git Action Gitlab...


In [4]:
df_resume.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [5]:
# delete unused columns
columns = ['Job_title','Job_description','skills','way_2_skills','combined']
df_skills  = df_skills[columns]
df_skills.head()

,Job_title,Job_description,skills,way_2_skills,combined
0,DevOps Engineer,We looking DevOps Engineer join team develop s...,"['engineer', 'unwanted traffic', 'infrastructu...","['System', 'Analyze', 'Policies', 'Debugging',...","['engineer', 'unwanted traffic', 'infrastructu..."
1,DevOps Engineer,Who We Are We owned funded one Egypt renowned ...,"['perspectives', 'strong focus scalability sec...","['Technical', 'Linux', 'Workflow', 'Lifecycle'...","['perspectives', 'strong focus scalability sec..."
2,DevOps Engineer,Wind IS Company Oracle Partner Egypt KSA UAE h...,"['development', 'engineer', 'Job Requirements ...","['System', 'Website', 'Design', 'Engineering',...","['development', 'engineer', 'Job Requirements ..."
3,Cloud Infrastructure Engineer,Who We Are We owned funded one Egypt renowned ...,"['related', 'perspectives', 'big stay', 'suit'...","['Hardware', 'Dhcp', 'Linux', 'Cloud', 'Vmware...","['related', 'perspectives', 'big stay', 'suit'..."
4,DevOps Engineer,About Axis We believe financial services acces...,['Jenkins CD Bambop CircleCI Git Action Gitlab...,"['Automation', 'Ansible', 'Docker', 'Security'...",['Jenkins CD Bambop CircleCI Git Action Gitlab...


In [6]:
count_title = df_skills['Job_title'].value_counts().reset_index()
number_of_title = len(df_skills['Job_title'].unique())
print('number of jobs  = ', number_of_title)
print('*'*50)
count_title

number of jobs  =  10271
**************************************************


,index,Job_title
0,Project Manager,519
1,Data Analyst,418
2,Software Developer,140
3,Software Engineer,120
4,Embedded Software Engineer,120
...,...,...
10265,Senior Hardware Design Engineer,1
10266,nd Line Engineer Rollout Deployment,1
10267,Hydrogen Electrical Design Engineer,1
10268,Process Safety Engineer Process Engineer,1


###### Data unbalanced so, i will use 25 most frequant

In [7]:
columns = count_title['index'][0:25]
columns = list(columns.values)
df_skills = df_skills[df_skills['Job_title'].isin(columns)]
df_skills

,Job_title,Job_description,skills,way_2_skills,combined
0,DevOps Engineer,We looking DevOps Engineer join team develop s...,"['engineer', 'unwanted traffic', 'infrastructu...","['System', 'Analyze', 'Policies', 'Debugging',...","['engineer', 'unwanted traffic', 'infrastructu..."
1,DevOps Engineer,Who We Are We owned funded one Egypt renowned ...,"['perspectives', 'strong focus scalability sec...","['Technical', 'Linux', 'Workflow', 'Lifecycle'...","['perspectives', 'strong focus scalability sec..."
2,DevOps Engineer,Wind IS Company Oracle Partner Egypt KSA UAE h...,"['development', 'engineer', 'Job Requirements ...","['System', 'Website', 'Design', 'Engineering',...","['development', 'engineer', 'Job Requirements ..."
4,DevOps Engineer,About Axis We believe financial services acces...,['Jenkins CD Bambop CircleCI Git Action Gitlab...,"['Automation', 'Ansible', 'Docker', 'Security'...",['Jenkins CD Bambop CircleCI Git Action Gitlab...
5,DevOps Engineer,Envision Employment Solutions currently look D...,"['DevOps Engineer role', 'repeat occurrences',...","['Automation', 'System', 'Docker', 'Engineerin...","['DevOps Engineer role', 'repeat occurrences',..."
...,...,...,...,...,...
15986,Quantity Surveyor,Apply A prestigious multi disciplinary constru...,"['full cost management role', 'development', '...","['Construction', 'Healthcare']","['full cost management role', 'development', '..."
16131,Software Engineer,Design develop test deploy maintain improve so...,"['development', 'Unix Linux', 'test', 'limited...","['Security', 'Design', 'Engineering', 'Coding'...","['development', 'Unix Linux', 'test', 'limited..."
16234,Software Engineer,Design develop test deploy maintain improve so...,"['development', 'Unix Linux', 'test', 'limited...","['Security', 'Design', 'Computer science', 'En...","['development', 'Unix Linux', 'test', 'limited..."
16285,Software Engineer,Design develop test deploy maintain improve so...,"['development', 'Python JavaScript Go Developm...","['Machine learning', 'Design', 'Engineering', ...","['development', 'Python JavaScript Go Developm..."


#`Data preprocessing`

In [8]:
def data_preprocessing(text):
  # remove emails
  text = re.sub(r'\S+@\S+', '', text)
  #remove links
  text = re.sub(r'http\S+','',text)
  #remove punctuation
  text = text.translate(str.maketrans('','',string.punctuation))
  #convert to lower
  text = text.lower()
  #remove digits and any thing unused
  text = ''.join([" " if ord(i)<32 or ord(i)>126 else i for i in text])
  text = ''.join([i for i in text if not i.isdigit()])
  #remove stop words
  text = ' '.join([i for i in text.split() if i not in stop_words])
  return text

In [9]:
df_resume['Resume'] = df_resume['Resume'].apply(data_preprocessing)

In [10]:
#combined skills
df_skills['combined'] = df_skills.apply(lambda row : ast.literal_eval(row['skills']) +ast.literal_eval(row['way_2_skills']),axis =1)
df_skills.head()

,Job_title,Job_description,skills,way_2_skills,combined
0,DevOps Engineer,We looking DevOps Engineer join team develop s...,"['engineer', 'unwanted traffic', 'infrastructu...","['System', 'Analyze', 'Policies', 'Debugging',...","[engineer, unwanted traffic, infrastructure vu..."
1,DevOps Engineer,Who We Are We owned funded one Egypt renowned ...,"['perspectives', 'strong focus scalability sec...","['Technical', 'Linux', 'Workflow', 'Lifecycle'...","[perspectives, strong focus scalability securi..."
2,DevOps Engineer,Wind IS Company Oracle Partner Egypt KSA UAE h...,"['development', 'engineer', 'Job Requirements ...","['System', 'Website', 'Design', 'Engineering',...","[development, engineer, Job Requirements BSc, ..."
4,DevOps Engineer,About Axis We believe financial services acces...,['Jenkins CD Bambop CircleCI Git Action Gitlab...,"['Automation', 'Ansible', 'Docker', 'Security'...",[Jenkins CD Bambop CircleCI Git Action Gitlab ...
5,DevOps Engineer,Envision Employment Solutions currently look D...,"['DevOps Engineer role', 'repeat occurrences',...","['Automation', 'System', 'Docker', 'Engineerin...","[DevOps Engineer role, repeat occurrences, clo..."


#Extract skills

In [11]:
df_skills = df_skills[df_skills['combined'].apply(len)>0]
df_skills= df_skills.reset_index(drop=True)

  ## Tricks (org)


In [12]:
# import spacy
# spacy_model = spacy.load('en_core_web_sm')
# doc = spacy_model('when i start learning Python and C++ i will learn c# and i love database , statistics and probability')
# for token in doc.ents:
#   print(token,' -->   ',token.label_)


Python  -->    ORG
C++  -->    LANGUAGE


In [13]:
# import spacy
# spacy_model = spacy.load('en_core_web_sm')
# text = df_skills['Job_description'][200]
# doc = spacy_model(text)
# lis=[]
# for token in doc.ents:
#   if token.label_ =='ORG':
#     lis.append(token)
# print(lis)

[AnalystThe Data Analyst, Key Responsibilities Collect, Import Clean Transform Validate Model, Qualifications University degree Computer Science Computer Applications Information Technology Information Systems, Hadoop Tableau Spotfire, SQL Datameer ETL, Python R, Knowledge Data, roadmap data, Qualifications Builds, Monitors, Integrates, Builds, Focus, BMO, Spotfire Tableau, Technical, Analytical, BMO Harris Bank, BMO Harris Bank, Harris Bank, Veteran]


### She use token.label_ as a directory for skills, which is incorrect

In [14]:
def extract_skills_1(resume_text):
  # Method 1 to extract skills (by spacy lib)
  doc = spacy_model(resume_text)
  spacy_skills = [ent.text for ent in doc.ents if ent.label_ in 'ORG']

  # Method 2 to extract skills (by nltk)
  nltk_token = word_tokenize(resume_text)
  # Use NLTK to extract keywords and phrases from resume text
  nltk_filtered = (token.lower() for token in nltk_token if token.lower() not in stop_words)
  nltk_keyword = nltk.FreqDist(nltk_filtered).most_common(50)
  # now he want to use grams
  nltk_bigram_measures = BigramAssocMeasures()
  nltk_finder = BigramCollocationFinder.from_words(nltk_filtered)
  nltk_finder.apply_freq_filter(1)
  nltk_phresas = nltk_finder.nbest(nltk_bigram_measures.raw_freq,50)

  # Skills
  nltk_skills  = list(set([keyword for keyword , freq in nltk_keyword] + [' '.join(phrase) for phrase in nltk_phresas]))
  skills  = list(set(spacy_skills + nltk_skills))
  return skills
resume_text = df_skills['Job_description'][200]
skills = extract_skills_1(resume_text)

print('Resume Text:', resume_text)
print('Skills:', skills)
# print('extra_Skills:', extra)
print('Skills len:', len(skills))
# print(job_data['Job_title'][100])

Resume Text: Address W Monroe S LaSalleJob Family Group Data Analytics ReportingData AnalystThe Data Analyst partner management business partners support development implementation advanced data analytical solutions drive measurable business outcomes create distinctive customer experience In role collaborate senior management dissect business problems conduct research analysis structure data build end end analytical solutions working multiple groups This role also assist design develop maintain analytical dashboards adhoc solutions support business driven decisions This includes preparation investigation data working closely Data Governance Technology teams supporting multiple lines business across enterprise help achieve objectives This role also needs take ownership data act SME developing automated data quality checks make sure quality data provided business consumption Key Responsibilities Collect business partner requirements data gathering integration enrichment Import Clean Tran

In [15]:
from pyresparser import ResumeParser
def extract_skills_2(text):
  #create doc
  doc = Document()
  doc.add_paragraph(text)
  doc.save('text.docx')
  data = ResumeParser('text.docx').get_extracted_data()
  skills = data.get('skills',[])
  return skills
resume_text = df_skills['Job_description'][200]
skills = extract_skills_2(resume_text)
print('Resume Text:', resume_text)
print('Skills:', skills)
# print('extra_Skills:', extra)
print('Skills len:', len(skills))
# print(job_data['Job_title'][100])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Resume Text: Address W Monroe S LaSalleJob Family Group Data Analytics ReportingData AnalystThe Data Analyst partner management business partners support development implementation advanced data analytical solutions drive measurable business outcomes create distinctive customer experience In role collaborate senior management dissect business problems conduct research analysis structure data build end end analytical solutions working multiple groups This role also assist design develop maintain analytical dashboards adhoc solutions support business driven decisions This includes preparation investigation data working closely Data Governance Technology teams supporting multiple lines business across enterprise help achieve objectives This role also needs take ownership data act SME developing automated data quality checks make sure quality data provided business consumption Key Responsibilities Collect business partner requirements data gathering integration enrichment Import Clean Tran

##` Example and How  [ResumeParser] works

In [16]:
# from pyresparser import ResumeParser

# def extract_skills(text):
#     doc = Document()
#     doc.add_paragraph(text)
#     doc.save('text2.docx')
#     data = ResumeParser('text2.docx').get_extracted_data()
#     skills = data.get('skills', [])
#     return skills
# # Example usage
# resume_text = """John Doe Skills: Python, Machine Learning, Natural Language Processing, Education: ...Experience: ..."""

# extracted_skills = extract_skills(resume_text)
# print("Extracted Skills:", extracted_skills)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Extracted Skills: ['Python', 'Machine learning']


why Extracted Skills: ['Python', 'Machine learning'] only ?
why not include Natural Language Processing


The extraction of skills from resumes using libraries like pyresparser is not perfect and may depend on various factors such as the structure of the resume, the way skills are mentioned, and the performance of underlying natural language processing (NLP) tools.



In [17]:
# # Save the updated data to a new CSV file
df_skills.to_csv('/content/drive/MyDrive/new_data_bygalal', index=False)

## Modeling

In [26]:
df_skills['combined'] = df_skills['combined'].apply(lambda row : ' '.join(row))
df_skills.head()

,Job_title,Job_description,skills,way_2_skills,combined
0,DevOps Engineer,We looking DevOps Engineer join team develop s...,"['engineer', 'unwanted traffic', 'infrastructu...","['System', 'Analyze', 'Policies', 'Debugging',...",engineer unwanted traffic infrastructure vulne...
1,DevOps Engineer,Who We Are We owned funded one Egypt renowned ...,"['perspectives', 'strong focus scalability sec...","['Technical', 'Linux', 'Workflow', 'Lifecycle'...",perspectives strong focus scalability security...
2,DevOps Engineer,Wind IS Company Oracle Partner Egypt KSA UAE h...,"['development', 'engineer', 'Job Requirements ...","['System', 'Website', 'Design', 'Engineering',...",development engineer Job Requirements BSc tech...
3,DevOps Engineer,About Axis We believe financial services acces...,['Jenkins CD Bambop CircleCI Git Action Gitlab...,"['Automation', 'Ansible', 'Docker', 'Security'...",Jenkins CD Bambop CircleCI Git Action Gitlab B...
4,DevOps Engineer,Envision Employment Solutions currently look D...,"['DevOps Engineer role', 'repeat occurrences',...","['Automation', 'System', 'Docker', 'Engineerin...",DevOps Engineer role repeat occurrences closel...


In [114]:
pip install imbalanced-learn

In [115]:
#import libs
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import OneHotEncoder ,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE


In [116]:
x = df_skills['combined']
y = df_skills['Job_title']
#use one-hot encoding to convert categorical labels to numerical form
encoder = LabelEncoder()
y_encoder = encoder.fit_transform(y.values.reshape(-1,1))
x_train,x_test,y_train,y_test = train_test_split(x,y_encoder,test_size = 0.2,random_state =123,stratify=y_encoder )

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##`Vectorization`
1. Tfidf
2. Word2Vec

# 1. `Tfidf`

In [119]:
vectorizer1 = TfidfVectorizer(stop_words='english',max_features=500)
x_train_tfidf = vectorizer1.fit_transform(x_train)
x_test_tfidf = vectorizer1.transform(x_test)


In [120]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(x_train_tfidf, y_train)

In [121]:
x_train_tfidf.toarray().shape , x_test_tfidf.toarray().shape ,y_train.shape

((1936, 500), (484, 500), (1936,))

### Build and Train the Random Forest Model


In [122]:
rf_classifier = RandomForestClassifier(class_weight = 'balanced',n_estimators=20,random_state=123)
#multi_output_classifier = MultiOutputClassifier(rf_classifier)
rf_classifier.fit(X_resampled,y_resampled)

RandomForestClassifier(class_weight='balanced', n_estimators=20,
                       random_state=123)

### Evaluate the Model

In [123]:
y_pred = rf_classifier.predict(x_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.76


In [107]:
y_test[0:5]

array([ 8, 19, 20, 22, 16])

In [108]:
y_pred[0:5]

array([ 8, 16, 23, 22, 16])

In [100]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.71      0.99      0.83        68
           2       1.00      0.43      0.60         7
           3       0.50      0.11      0.18         9
           4       0.67      0.50      0.57         8
           5       0.91      1.00      0.96        32
           6       0.75      0.67      0.71         9
           7       0.50      0.14      0.22         7
           8       0.86      0.86      0.86         7
           9       0.89      0.94      0.92        18
          10       1.00      0.90      0.95        10
          11       0.89      1.00      0.94         8
          12       0.00      0.00      0.00         4
          13       0.75      0.75      0.75         8
          14       0.83      0.45      0.59        11
          15       0.75      0.23      0.35        13
          16       0.67      0.91      0.77       109
          17       0.88    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Fine-Tuning with Grid Search

In [57]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_classifier = RandomForestClassifier(random_state=42)
grid_search  = GridSearchCV(rf_classifier,param_grid,cv=3,scoring = 'accuracy')
grid_search.fit(x_train_tfidf,y_train)
best_parms = grid_search.best_params_
print(f'Best Hyperparameters: {best_parms}')

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [58]:
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(x_test_tfidf)
# Evaluate accuracy
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Accuracy with Best Model: {accuracy_best:.2f}')

# Print classification report for the best model
print(classification_report(y_test, y_pred_best))

Accuracy with Best Model: 0.76
                            precision    recall  f1-score   support

           Account Manager       1.00      0.70      0.82        10
              Data Analyst       0.74      1.00      0.85        68
             Data Engineer       1.00      0.29      0.44         7
            Data Scientist       1.00      0.33      0.50         9
           DevOps Engineer       0.88      0.88      0.88         8
Embedded Software Engineer       0.94      1.00      0.97        32
           Finance Manager       1.00      0.44      0.62         9
        IT Project Manager       0.00      0.00      0.00         7
       IT Support Engineer       1.00      1.00      1.00         7
     Management Accountant       0.78      1.00      0.88        18
         Marketing Manager       1.00      1.00      1.00        10
Mechanical Design Engineer       1.00      1.00      1.00         8
         Programme Manager       0.00      0.00      0.00         4
     Project Adm

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##`doc2vec`

In [124]:
# Define a function to convert each document into a TaggedDocument
def tag_docs(docs):
  tagged_docs = []
  for i,doc in enumerate(docs):
    tagged_doc = TaggedDocument(words = doc,tags=[i])
    tagged_docs.append(tagged_doc)
  return tagged_docs


In [125]:
tagged_docs_combined = tag_docs(df_skills['combined'])
tagged_docs_combined[0]

TaggedDocument(words='engineer unwanted traffic infrastructure vulnerabilities application deployment issues team best software Requirements Azure AWS GCP certifications products services computer system environment security accordance various operational issues computer systems experience operational develop solutions IT team Requirements Azure AWS GCP DevOps Engineer responsibilities DevOps Engineer join team monitoring Responsibilities Drain traffic away cluster Roll back bad software Responsibilities Drain issues Use monitoring systems Identify practices traffic computer systems building info computer efficient functioning data storage process functions accordance company security policies organization software systems automated solutions operational aspects organization DevOps Engineer responsibilities additional serving capacity We systems ensure cloud solutions accordance best practices dashboards Administer production jobs Block rate cloud devops System Analyze Policies Debuggi

In [126]:
model_doc2vec = Doc2Vec.load('/content/drive/MyDrive/model_doc2vec_new.model')

In [127]:
jop_vectors = np.array([model_doc2vec.dv[i] for i in range(len(model_doc2vec.dv))])
jop_vectors

array([[-0.5729557 , -0.31342244,  0.17791623, ...,  0.43079573,
         1.4437256 , -0.26233485],
       [ 0.8104713 ,  0.41525948,  0.5110505 , ..., -0.89782   ,
         1.5060788 ,  0.09355199],
       [ 0.27690735, -0.17167495,  0.48103216, ...,  0.10354999,
         0.6545089 ,  0.69339216],
       ...,
       [-0.08073098,  0.8041919 ,  0.42424902, ...,  0.25474682,
         0.9379102 ,  0.3687203 ],
       [-0.09911393,  0.43033364,  0.31832436, ...,  0.26819018,
         0.6657692 ,  0.51827735],
       [-0.08631754, -0.4320736 ,  0.22695276, ...,  0.0048679 ,
         0.18077406, -0.10587844]], dtype=float32)

In [131]:
x = jop_vectors
y = df_skills['Job_title']
#use one-hot encoding to convert categorical labels to numerical form
encoder = OneHotEncoder()
y_onehot = encoder.fit_transform(y.values.reshape(-1,1))
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state =123)

In [132]:
x_train_tfidf.toarray().shape , x_test_tfidf.toarray().shape

((1936, 500), (484, 500))

In [139]:

rf_classifier = RandomForestClassifier(class_weight='balanced',n_estimators=50,random_state=123)
#multi_output_classifier = MultiOutputClassifier(rf_classifier)
rf_classifier.fit(x_train_tfidf,y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=50,
                       random_state=123)

In [140]:
y_pred = rf_classifier.predict(x_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.18


In [141]:

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_classifier = RandomForestClassifier(random_state=42)
grid_search  = GridSearchCV(rf_classifier,param_grid,cv=3,scoring = 'accuracy')
grid_search.fit(x_train_tfidf,y_train)
best_parms = grid_search.best_params_
print(f'Best Hyperparameters: {best_parms}')

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [142]:
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(x_test_tfidf)
# Evaluate accuracy
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Accuracy with Best Model: {accuracy_best:.2f}')

# Print classification report for the best model
print(classification_report(y_test, y_pred_best))


Accuracy with Best Model: 0.20
                            precision    recall  f1-score   support

           Account Manager       0.00      0.00      0.00        10
              Data Analyst       0.12      0.19      0.14        68
             Data Engineer       0.00      0.00      0.00         7
            Data Scientist       0.00      0.00      0.00         9
           DevOps Engineer       0.00      0.00      0.00         8
Embedded Software Engineer       0.00      0.00      0.00        32
           Finance Manager       0.00      0.00      0.00         9
        IT Project Manager       0.00      0.00      0.00         7
       IT Support Engineer       0.00      0.00      0.00         7
     Management Accountant       0.00      0.00      0.00        18
         Marketing Manager       0.00      0.00      0.00        10
Mechanical Design Engineer       0.00      0.00      0.00         8
         Programme Manager       0.00      0.00      0.00         4
     Project Adm

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
